In [1]:
import pyvips
import os

src = '/Volumes/Digital pathology image lib/JHU/Laura Wood/IF for SenPan001/FWPanc001_207/1st round/well image stitch/1x_'
ims = [_ for _ in os.listdir(src) if _.endswith('tif')]

In [2]:
im1 = pyvips.Image.new_from_file(os.path.join(src,ims[0]))
im2 = pyvips.Image.new_from_file(os.path.join(src,ims[1]))
im3 = pyvips.Image.new_from_file(os.path.join(src,ims[2]))
im4 = pyvips.Image.new_from_file(os.path.join(src,ims[3]))
im = im1.bandjoin([im2,im3,im4])

In [4]:
image_height = im.height
image_bands = im.bands

# split to separate image planes and stack vertically ready for OME
im = pyvips.Image.arrayjoin(im.bandsplit(), across=1)

In [5]:
# set minimal OME metadata
# before we can modify an image (set metadata in this case), we must take a
# private copy
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")

im.tiffsave(os.path.join(src,'multi_channel_pyramid.ome.tiff'), compression="jpeg", tile=True,tile_width=512, tile_height=512,pyramid=True, subifd=True)